In [52]:
#!/usr/bin/env python
%matplotlib inline

from PyOPC.OPCContainers import *
from PyOPC.XDAClient import XDAClient

import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sqlalchemy import create_engine # database connection
import datetime as dt
from IPython.display import display
from IPython.display import clear_output

import plotly.plotly as py # interactive graphing
from plotly.graph_objs import Bar, Scatter, Marker, Layout 

#http://pyopc.sourceforge.net/
#https://plot.ly/python/big-data-analytics-with-pandas-and-sqlite/

In [53]:
address=''
%run 'address'  #lese geheime address='http://ip-number:port'

In [54]:
xda = XDAClient(OPCServerAddress=address,
                ReturnErrorText=True)

In [55]:
path='Loc/Wec/Plant2'

In [63]:
disk_engine = create_engine('sqlite:///egneos1.db')

In [ ]:
while 1:
    itemcollect=[]
    data={}
    for i in xda.Browse(ItemName=path)[0][:]:
        #print i.Name
        if not i.HasChildren:
            i = ItemContainer(ItemName=path+"/"+i.Name)
            itemcollect.append(i)
    #print itemcollect
    clear_output()
    %time all=xda.Read(itemcollect)
    
    data['Timestamp']=all[1]['RcvTime']

    for i in all[0]:
        i.Name=str.replace(i.ItemName,path+"/","")
        #print i.Name,"  ",i.Value
        data[i.Name]=i.Value

    data
    series=pd.Series(data)
    pd.DataFrame(series).T.to_sql('data3', disk_engine, if_exists='append')
    time.sleep(1)

In [39]:
pd.DataFrame(series).T.to_sql('data3', disk_engine, if_exists='append')

In [ ]:
df = pd.read_sql_query('SELECT * FROM data3 LIMIT 300', disk_engine)
df

In [43]:
pd.DataFrame(data=series).T

,Activ,Alias,CosPhi,Freq,GoPos,Hour,I-L1,I-L2,I-L3,NRotor,...,SerieNo,Timestamp,TypeNo,TypeStr,U-L1,U-L2,U-L3,Vane,Vwind,Wexp
0,True,,0.996133,49.99,134,1946,1082,1085,1052,10.23,...,1011128,2016-02-07 12:54:06,241,CS101a,242,241,242,-12.9,7.3,2385792


In [49]:
series

Activ                        True
Alias                            
CosPhi                    0.99834
Freq                        50.01
GoPos                         136
Hour                         1947
I-L1                          627
I-L2                          634
I-L3                          640
NRotor                       8.76
NomP                         3075
P                             468
PavaExtern                    463
PavaForceM                    463
PavaTech                      463
PavaVWind                     463
Q                             -27
SerieNo                   1011128
Timestamp     2016-02-07 13:18:25
TypeNo                        241
TypeStr                    CS101a
U-L1                          242
U-L2                          242
U-L3                          242
Vane                         -5.9
Vwind                         5.5
Wexp                      2386055
dtype: object